In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import math  # ceil 사용을 위해 추가


# 1) 하이퍼파라미터 설정

batch_size = 64            # 한 배치에 처리할 이미지 수
epochs = 50                # 전체 학습 반복 횟수
learning_rate = 0.001      # 초기 학습률
input_size = 128           # 이미지 가로 픽셀 수 (RNN input feature 차원)
sequence_length = 128      # 이미지 세로 픽셀 수 (RNN time step 수)
hidden_size = 256          # RNN 은닉 상태 크기
num_layers = 3             # RNN 레이어 개수 (stacked RNN)
num_classes = 4            # 분류할 클래스 개수 (종양 종류 수)


# 2) 데이터 증강 및 전처리 설정

data_dir = '/content/BrainMRI'
assert os.path.exists(data_dir), "데이터 폴더가 존재하지 않습니다."

# ImageDataGenerator를 이용한 데이터 증강 및 정규화
train_gen = ImageDataGenerator(
    rescale=1./255,            # 픽셀 값을 [0,1] 범위로 정규화
    validation_split=0.3,      # 전체 데이터의 30%를 검증용으로 분리
    rotation_range=30,         # 랜덤 회전 최대 30도
    zoom_range=0.2,            # 랜덤 줌 최대 20%
    width_shift_range=0.2,     # 좌우 이동 최대 20%
    height_shift_range=0.2,    # 상하 이동 최대 20%
    horizontal_flip=True       # 좌우 반전 허용
)

val_gen = ImageDataGenerator(
    rescale=1./255,            # 검증 데이터는 증강 없이 정규화만 적용
    validation_split=0.3
)

# 증강 포함 학습 데이터 생성기
train_generator = train_gen.flow_from_directory(
    data_dir,
    target_size=(input_size, sequence_length),  # 이미지 크기 맞춤 (128x128)
    color_mode='grayscale',                      # 흑백 이미지 (1채널)
    batch_size=batch_size,
    class_mode='categorical',                     # 다중 클래스 원-핫 인코딩 라벨
    subset='training',                            # 학습용 데이터 분할
    shuffle=True                                 # 매 epoch마다 셔플
)

# 검증 데이터 생성기
val_generator = val_gen.flow_from_directory(
    data_dir,
    target_size=(input_size, sequence_length),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',                          # 검증용 데이터 분할
    shuffle=False                                # 검증 시 셔플 불필요
)


# 3) RNN 기반 모델 구성

# 입력 형태: (batch_size, sequence_length=128, input_size=128, 채널=1)
inputs = layers.Input(shape=(sequence_length, input_size, 1))

# 4차원 텐서에서 채널 차원 제거하여 (batch, 128, 128) 형태로 변환
x = layers.Reshape((sequence_length, input_size))(inputs)

# stacked RNN 구성: 3개의 SimpleRNN 레이어 순차적 연결
for i in range(num_layers):
    # 마지막 RNN 레이어는 마지막 time step의 은닉 상태만 반환(return_sequences=False)
    return_seq = (i != num_layers - 1)
    x = layers.SimpleRNN(hidden_size, return_sequences=return_seq)(x)

# 마지막 RNN 출력에 fully connected layer 연결하여 클래스 확률 출력
outputs = layers.Dense(num_classes, activation='softmax')(x)

# 모델 생성
model2 = models.Model(inputs, outputs)

# 모델 컴파일: Adam 옵티마이저 + 다중 클래스 교차엔트로피 + 정확도 평가
model2.compile(
    optimizer=optimizers.Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 모델 구조 요약 출력
model2.summary()


# 4) 콜백 설정: 학습 과정 제어

earlystop = callbacks.EarlyStopping(
    monitor='val_loss',          # 검증 손실 기준 조기 종료
    patience=5,                  # 5회 연속 개선 없으면 종료
    restore_best_weights=True    # 최적 가중치 복원
)

checkpoint = callbacks.ModelCheckpoint(
    'best_model.keras',          # 모델 저장 경로
    monitor='val_accuracy',      # 검증 정확도 기준 저장
    save_best_only=True          # 가장 좋은 모델만 저장
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',          # 검증 손실 기준 학습률 감소
    factor=0.5,                 # 학습률 50% 감소
    patience=2,                 # 2회 연속 개선 없으면 감소
    verbose=1
)


# 5) 모델 학습 실행

steps_per_epoch = math.ceil(train_generator.samples / batch_size)      # 총 학습 샘플 수에 따라 epoch당 스텝 계산 (올림)
validation_steps = math.ceil(val_generator.samples / batch_size)       # 검증 샘플 기준 스텝 계산 (올림)

history = model2.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    steps_per_epoch=steps_per_epoch,       # 한 epoch당 학습 스텝 수
    validation_steps=validation_steps,     # 검증 시 스텝 수
    callbacks=[earlystop, checkpoint, reduce_lr]  # 콜백 적용
)


# 6) 학습 결과 시각화

plt.figure(figsize=(12, 5))

# 손실 곡선 시각화
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# 정확도 곡선 시각화
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy Curve')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()
